In [4]:
import numpy as np
import random
from pymoo.core.problem import Problem
from pymoo.problems import get_problem
from pymoo.util.plotting import plot
from pymoo.util.ref_dirs import get_reference_directions
from pymoo.visualization.scatter import Scatter

from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.optimize import minimize


In [ ]:
from src.multi_evolutive import RealMultiEvolutive
from pymoo.problems import get_problem
from pymoo.util.plotting import plot
import matplotlib.pyplot as plt
import numpy as np
from src.evaluation import MultiObjectiveEvaluation
problem = get_problem("zdt1")
evaluation = MultiObjectiveEvaluation(
    evaluate=problem.evaluate,
    problem=problem,
    reference_point=(1., 1.),
    # limits=[[0, 1], [0, 1]]
)
evolutive = RealMultiEvolutive(
    n_individuals=50,
    value_range=[0, 1],
    maximize=[False, False],
    cromolength=30,
    mutation=["random-range", "random-local"],
    crossover=["blx", "uniform", "blx-alpha"],
    front='front',
    penalization='crowding_norm',
    selection_pool_size=0.9,
    average_mutation_rate=1.25,
    mutation_eps=0.5,
    elitism=True,
    alpha=0.2,
    T_selection=6,
    evaluation_metrics=evaluation
)

results = evolutive.evolve(fit=problem.evaluate, n_generations=200, trace=50, obtain_metrics=True)
evolutive.plot_evolution_metrics(results["evolution_metrics"])
plt.show()

fig, axs = plt.subplots(1, 1)
axs.set_ylim(-0.25, 1.25)
pareto = problem.pareto_front()
axs.scatter(pareto[:, 0], pareto[:, 1], label='pareto')
fit_scores = np.array([problem.evaluate(p) for p in results["best"]])

axs.scatter(fit_scores[:, 0], fit_scores[:, 1], label='obtained')
plt.legend()
# plot(problem.evaluate(results["best"]))


In [ ]:
scatter = Scatter()

algorithm = NSGA2(pop_size=100)
res = minimize(problem,
               algorithm,
               ('n_gen', 100),
               verbose=False)
scatter.add(problem.pareto_front(), label="Pareto optimo")
scatter.add(res.F, label="NSGA2_100gen")
scatter.legend=True
scatter.show()

In [ ]:
fig, axs = plt.subplots(1, 1)
pareto = problem.pareto_front()
axs.scatter(pareto[:, 0], pareto[:, 1], label='pareto')
fit_scores = np.array([problem.evaluate(p) for p in results["best"]])

axs.scatter(fit_scores[:, 0], fit_scores[:, 1], label='obtained')
plt.legend()

In [ ]:
problem = get_problem("zdt3")
evolutive = RealMultiEvolutive(
    n_individuals=50,
    value_range=[0, 1],
    maximize=[False, False],
    cromolength=30,
    mutation=["random-range", "random-local"],
    crossover=["blx", "uniform", "blx-alpha"],
    front='front',
    penalization='crowding_norm',
    selection_pool_size=0.8,
    average_mutation_rate=1.5,
    mutation_eps=0.5,
    elitism=True,
    alpha=0.6,
    T_selection=10
)

results = evolutive.evolve(fit=problem.evaluate, n_generations=200, trace=50, obtain_metrics=True)
evolutive.plot_evolution_metrics(results["evolution_metrics"])
plt.show()

fig, axs = plt.subplots(1, 1)
# axs.set_ylim(-1.5, 1.5)
pareto = problem.pareto_front()
axs.scatter(pareto[:, 0], pareto[:, 1], label='pareto')
fit_scores = np.array([problem.evaluate(p) for p in results["best"]])

axs.scatter(fit_scores[:, 0], fit_scores[:, 1], label='obtained')
plt.legend()

In [ ]:
scatter = Scatter()

algorithm = NSGA2(pop_size=100)
res = minimize(problem,
               algorithm,
               ('n_gen', 100),
               verbose=False)
scatter.add(problem.pareto_front(), label="Pareto optimo")
scatter.add(res.F, label="NSGA2_100gen")
scatter.legend=True
scatter.show()

In [ ]:
problem = get_problem("mw7")
evolutive = RealMultiEvolutive(
    n_individuals=100,
    value_range=[0, 1],
    maximize=[False, False],
    cromolength=30,
    mutation=["random-range", "random-local"],
    crossover=["blx", "uniform", "blx-alpha"],
    front_type='range',
    niche_type='sharing',
    selection_pool_size=0.6,
    average_mutation_rate=1.,
    mutation_eps=0.05,
    elitism=True,
    alpha=0.1,
    T_selection=4
)
results = evolutive.evolve(fit=problem.evaluate, n_generations=300, trace=100, obtain_metrics=True)
evolutive.plot_evolution_metrics(results["evolution_metrics"])
plt.show()

fig, axs = plt.subplots(1, 1)
# axs.set_ylim(-0.25, 1.25)
pareto = problem.pareto_front()
axs.scatter(pareto[:, 0], pareto[:, 1], label='pareto')
fit_scores = np.array([problem.evaluate(p) for p in results["best"]])

axs.scatter(fit_scores[:, 0], fit_scores[:, 1], label='obtained')
plt.legend()

In [ ]:
ref_dirs = get_reference_directions("das-dennis", 3, n_partitions=12)
problem = get_problem("mw14").pareto_front()
Scatter(angle=(45,45)).add(problem, color="red").show()

In [5]:
class TSPProblem(Problem):
    def __init__(self, n_cities, seed=50):
        # Define the number of variables (cities) and objectives (time and distance)
        super().__init__(n_var=n_cities, n_obj=2, n_constr=0, xl=0, xu=1)
        
        # Initialize the random seed for reproducibility
        if seed is not None:
            np.random.seed(seed)
        
        # Generate random distances and times between cities
        self.distances = np.random.uniform(0, 100, size=(n_cities, n_cities))
        self.distances = (self.distances + self.distances.T) / 2
        self.times = np.random.uniform(0, 300, size=(n_cities, n_cities))
        random.seed(None)
        
        # Ensure the diagonal is zero (distance/time from a city to itself is zero)
        np.fill_diagonal(self.distances, 0)
        np.fill_diagonal(self.times, 0)

    def _evaluate(self, x, out, *args, **kwargs):
        # Calculate the total distance and time for each solution in x
        distances = []
        times = []
        
        for route in x:
            perm = np.argsort(route)
            total_distance = 0
            total_time = 0
            
            for i in range(len(perm) - 1):
                total_distance += self.distances[perm[i], perm[i+1]]
                total_time += self.times[perm[i], perm[i+1]]
            
            # Add distance and time from last city back to the first city
            total_distance += self.distances[perm[-1], perm[0]]
            total_time += self.times[perm[-1], perm[0]]
            
            distances.append(total_distance)
            times.append(total_time)
        
        out["F"] = np.column_stack([distances, times])

Comprobación del funcionamiento del algoritmo NSGA2 para las funciones continuas

In [ ]:
problem = get_problem("mw14", n_var = 30)
print(problem.n_var)
print(problem.n_obj)
print(problem.xl)
print(problem.xu)


In [11]:
algorithm = NSGA2(pop_size=100)
res = minimize(problem,
               algorithm,
               ('n_gen', 100),
               verbose=False)

In [ ]:
scatter = Scatter()
scatter.add(problem.pareto_front(), label="Pareto optimo")
scatter.add(res.F, label="NSGA2_100gen")
scatter.legend=True
scatter.show()

Comprobación del funcionamiento del algoritmo NSGA2 para el TSPMO

In [ ]:
problem = TSPProblem(100)
print(np.shape(problem.distances))
print(np.shape(problem.times))

In [9]:
algorithm = NSGA2(pop_size=100)
res = minimize(problem,
               algorithm,
               ('n_gen', 10),
               verbose=False)
res2 = minimize(problem,
               algorithm,
               ('n_gen', 1000),
               verbose=False)

In [ ]:
scatter = Scatter()
scatter.add(res.F, label="NSGA2_10")
scatter.add(res2.F, label="NSGA2_1000")
scatter.legend=True
scatter.show()


In [ ]:
from src.multi_evolutive import PermutationMultiEvolutive
import matplotlib.pyplot as plt

evolutive = PermutationMultiEvolutive(
    alphabet=range(100),
    n_individuals=50,
    mutation=["to-opt", "insert-gene", "swap-gene"],
    crossover= ["uniform", "split-merge"],
    front='front',
    penalization='crowding_norm',
    selection_pool=['best', 'section'],
	T_selection=20,
	p_crossover=0.7,
	average_mutation_rate=1.25,
    maximize=[False, False],
    selection_pool_size=1.5,
	elitism=True
)
results = evolutive.evolve(fit=problem.evaluate, n_generations=2000, trace=400, obtain_metrics=True)
fig, axs = plt.subplots(1, 1)
axs.scatter(res2.F[:, 0], res2.F[:, 1], label='nsga2')
fit_scores = np.array([problem.evaluate(p) for p in results["best"]])

axs.scatter(fit_scores[:, 0], fit_scores[:, 1], label='obtained')
plt.legend()

In [ ]:
from src.multi_evolutive import RealMultiEvolutive
import matplotlib.pyplot as plt
import numpy as np

evolutive = RealMultiEvolutive(
    n_individuals=50,
    value_range=[0, 1],
    maximize=[False, False],
    cromolength=100,
    mutation=["random-range", "random-local"],
    crossover=["blx", "uniform", "blx-alpha", "split-merge"],
    front='front',
    penalization='crowding_norm',
    selection_pool=[('best', 1), ('section', 2)], # , ('best', 4)],
    selection_pool_size=1.,
    average_mutation_rate=1.,
    mutation_eps=.5,
    elitism=True,
    alpha=0.6,
    T_selection=10
)
results = evolutive.evolve(fit=problem.evaluate, n_generations=2000, trace=500, obtain_metrics=True)
fig, axs = plt.subplots(1, 1)
axs.scatter(res2.F[:, 0], res2.F[:, 1], label='nsga2')
fit_scores = np.array([problem.evaluate(p) for p in results["best"]])

axs.scatter(fit_scores[:, 0], fit_scores[:, 1], label='obtained')
plt.legend()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from src.utils import cluster_points
# Parameters
num_point_clouds = 10
base_num_points = 1000  # Base number of points per cloud
variation = 100         # Allowable variation in the number of points

# Generate random point clouds
point_clouds = [
    np.random.rand(np.random.randint(base_num_points - variation, base_num_points + variation), 2)
    for _ in range(num_point_clouds)
]
res_lens = [len(res) for res in point_clouds]
k_cluster_points = min(res_lens)
mean_points = sum(res_lens) / len(point_clouds)
print(k_cluster_points, len(point_clouds[0][0]))
mean_point_front = np.zeros((k_cluster_points, len(point_clouds[0][0])))
plt.subplots(1, 1)
for idx, pc in enumerate(point_clouds):
    plt.scatter(pc[:, 0], pc[:, 1], label=idx)
    mean_point_front += cluster_points(pc, k_cluster_points)
mean_point_front /= len(point_clouds)
plt.scatter(mean_point_front[:, 0], mean_point_front[:, 1], label='mean')
plt.legend()
plt.show()

In [5]:
from examples.tsp_multi import TSPProblem
from src.multi_evolutive import RealMultiEvolutive
from src.evaluation import GridSearch, MultiObjectiveEvaluation

problem = TSPProblem(100)
evolution_metric = MultiObjectiveEvaluation(
    evaluate=problem.evaluate,
    reference_point=(15000, 6000),
)
static_evolution_kwargs ={
    "value_range": [0, 1],
    "maximize": [False, False],
    "cromolength": 100,
    "front": 'front',
    "penalization": 'crowding_norm',
    "selection_pool_size": 1.,
    "elitism":True,
    "T_selection": 8
}
evolution_kwargs = {
    "mutation": ["random-range", "random-local"],
    "crossover": ["blx", "uniform", "blx-alpha", "split-merge"],
    "selection_pool": [(('best', 1), ('section', 2)), ('best', 'roulette'), (('roulette', 1), ('section', 2))],
    "average_mutation_rate": [1., 1.25],
    "mutation_eps": [0.5, 0.75],
    "alpha": [0.4, 0.6],
}

grid = GridSearch(
    save_file_name='./evaluations/tsp_multi',
    result_names='tspm',
    fit=problem.evaluate,
    n_individuals=50,
    n_generations=2000,
    n_iterations=10,
    Evolution=RealMultiEvolutive,
    static_evolution_kwargs=static_evolution_kwargs,
    evolution_kwargs=evolution_kwargs,
    evolution_metric=evolution_metric,
    exclude_combinations_list=(1,)
)

Number of Evaluations: 264
Current results: 1


In [7]:
grid.search()

Evaluation 145: {'mutation': ['random-range', 'random-local'], 'crossover': ('blx', 'uniform', 'blx-alpha'), 'selection_pool': (('best', 1), ('section', 2)), 'average_mutation_rate': 1.0, 'mutation_eps': 0.5, 'alpha': 0.6, 'value_range': [0, 1], 'maximize': [False, False], 'cromolength': 100, 'front': 'front', 'penalization': 'crowding_norm', 'selection_pool_size': 1.0, 'elitism': True, 'T_selection': 8}


KeyboardInterrupt: 